# Projet Createur de Mail personnalise

Ce notebook illustre un workflow multi-agents pour creer des emails personnalises. Deux agents collaborent :
- **InputCollector** : Collecte les informations necessaires via des questions interactives
- **EmailGenerator** : Genere le brouillon d'email a partir des informations collectees

Le notebook utilise :
- **Semantic Kernel** pour l'orchestration des agents
- **Pydantic** pour la gestion de l'etat partage (`EmailState`)
- **ipywidgets** pour l'interface utilisateur interactive

In [ ]:
# Cellule 1: Installations et Imports
# Installations et Imports
%pip install --quiet semantic-kernel python-dotenv pydantic ipywidgets

import os
from dotenv import load_dotenv
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from pydantic import BaseModel
import asyncio
import ipywidgets as widgets
from IPython.display import display, clear_output

# Chargement des variables d'environnement depuis le fichier .env parent
# Le fichier .env doit contenir OPENAI_API_KEY et optionnellement OPENAI_MODEL_ID, OPENAI_BASE_URL
load_dotenv('../.env')

## 1. Installations et Imports

Cette cellule installe les dependances necessaires :
- **semantic-kernel** : SDK pour l'orchestration des agents LLM
- **python-dotenv** : Chargement securise des variables d'environnement
- **pydantic** : Validation et serialisation des donnees
- **ipywidgets** : Widgets interactifs pour Jupyter

Le fichier `.env` parent doit contenir `OPENAI_API_KEY` et optionnellement `OPENAI_MODEL_ID` et `OPENAI_BASE_URL`.

## 2. Configuration OpenAI et definition de l'etat

Cette section definit :
- **add_openai_service()** : Configure le service OpenAI avec support pour endpoints personnalises (LM Studio, Ollama)
- **EmailState** : Classe Pydantic qui maintient l'etat de l'email en cours de creation

L'etat partage permet aux deux agents de collaborer sur les memes donnees.

In [ ]:
# Cellule 2: Configuration OpenAI et définition de l'état

# Chargement des variables d'environnement depuis le fichier .env parent
load_dotenv('../.env')

# Configuration de l'API OpenAI (lecture depuis .env)
def add_openai_service(kernel):
    api_key = os.getenv("OPENAI_API_KEY")
    model_id = os.getenv("OPENAI_MODEL_ID", "gpt-4o-mini")
    base_url = os.getenv("OPENAI_BASE_URL")

    if not api_key:
        raise ValueError("OPENAI_API_KEY non definie dans le fichier .env")

    service_kwargs = {
        "service_id": "default",
        "ai_model_id": model_id,
        "api_key": api_key
    }

    # Support pour endpoint personnalise (ex: LM Studio, Ollama)
    if base_url:
        from openai import AsyncOpenAI
        async_client = AsyncOpenAI(api_key=api_key, base_url=base_url)
        service_kwargs["async_client"] = async_client

    kernel.add_service(OpenAIChatCompletion(**service_kwargs))

# Définition de EmailState
class EmailState(BaseModel):  # Hérite de BaseModel pour la sérialisation
    type: str = ""  # Type d'email (professionnel, amical...)
    recipient_name: str = ""
    recipient_role: str = ""
    subject: str = ""
    key_points: list[str] = []
    tone: str = ""  # Formel, informel, etc.
    draft: str = ""  # Brouillon de l'email
    persona: str = ""  # Persona sélectionné
    is_complete: bool = False  # Indique si l'email est prêt à être généré
    human_input_requested: bool = False  # Flag pour indiquer qu'une entrée utilisateur est attendue
    current_question: str = ""  # Question actuellement posée à l'utilisateur
    conversation_complete: bool = False  # Indique si la conversation est terminée

## 3. Plugin InputCollector

Le plugin `InputCollectorPlugin` expose des fonctions kernel que l'agent peut appeler :
- **set_type**, **set_recipient_name**, etc. : Enregistrent les informations collectees
- **get_state_summary** : Resume l'etat actuel pour verification
- **check_completeness** : Verifie si toutes les informations requises sont presentes
- **ask_human** : Signale qu'une question doit etre posee a l'utilisateur

Chaque fonction est decoree avec `@kernel_function` pour etre exposee au LLM.

In [3]:
# Cell 3: Plugin InputCollector

from semantic_kernel.functions import kernel_function

class InputCollectorPlugin:
    def __init__(self, state: EmailState):
        self.state = state

    @kernel_function(
        name="set_type",
        description="Définit le type d'email (professionnel, amical, etc.)"
    )
    def set_type(self, type: str) -> str:
        self.state.type = type
        return f"Type d'email défini sur {type}"

    @kernel_function(
        name="set_recipient_name",
        description="Définit le nom du destinataire"
    )
    def set_recipient_name(self, recipient_name: str) -> str:
        self.state.recipient_name = recipient_name
        return f"Nom du destinataire défini sur {recipient_name}"

    @kernel_function(
        name="set_recipient_role",
        description="Définit le rôle du destinataire"
    )
    def set_recipient_role(self, recipient_role: str) -> str:
        self.state.recipient_role = recipient_role
        return f"Rôle du destinataire défini sur {recipient_role}"

    @kernel_function(
        name="set_subject",
        description="Définit le sujet de l'email"
    )
    def set_subject(self, subject: str) -> str:
        self.state.subject = subject
        return f"Sujet de l'email défini sur {subject}"

    @kernel_function(
        name="add_key_point",
        description="Ajoute un point clé à aborder dans l'email"
    )
    def add_key_point(self, key_point: str) -> str:
        self.state.key_points.append(key_point)
        return f"Point clé ajouté : {key_point}"

    @kernel_function(
        name="set_tone",
        description="Définit le ton de l'email (formel, informel, etc.)"
    )
    def set_tone(self, tone: str) -> str:
        self.state.tone = tone
        return f"Ton de l'email défini sur {tone}"

    @kernel_function(
        name="get_state_summary",
        description="Obtient un résumé de l'état actuel de l'email"
    )
    def get_state_summary(self) -> str:
        summary = "État actuel de l'email :\n"
        summary += f"- Type: {self.state.type or 'Non défini'}\n"
        summary += f"- Destinataire: {self.state.recipient_name or 'Non défini'}"
        if self.state.recipient_role:
            summary += f" ({self.state.recipient_role})"
        summary += f"\n- Sujet: {self.state.subject or 'Non défini'}\n"
        summary += f"- Points clés: {', '.join(self.state.key_points) if self.state.key_points else 'Aucun'}\n"
        summary += f"- Ton: {self.state.tone or 'Non défini'}\n"
        return summary

    @kernel_function(
        name="check_completeness",
        description="Vérifie si toutes les informations nécessaires sont disponibles"
    )
    def check_completeness(self) -> str:
        required_fields = ["type", "recipient_name", "subject", "tone"]
        missing_fields = []

        for field in required_fields:
            if not getattr(self.state, field):
                missing_fields.append(field)

        if not self.state.key_points:
            missing_fields.append("key_points")

        if missing_fields:
            return f"Informations manquantes: {', '.join(missing_fields)}"
        else:
            self.state.is_complete = True
            return "Toutes les informations nécessaires sont disponibles."

    @kernel_function(
        name="ask_human",
        description="Pose une question à l'utilisateur humain et attend sa réponse"
    )
    def ask_human(self, question: str) -> str:
        # Cette fonction signale simplement que l'entrée humaine est nécessaire
        # L'implémentation réelle se fait dans la boucle principale
        self.state.human_input_requested = True
        self.state.current_question = question
        return f"Question posée à l'utilisateur: {question}"

## 4. Plugin EmailGenerator

Le plugin `EmailGeneratorPlugin` contient la logique de generation d'email :
- **generate_draft()** : Construit un prompt a partir de l'etat et appelle le LLM pour generer le brouillon
- Le prompt integre tous les parametres : type, destinataire, sujet, points cles, ton, et optionnellement un persona

La methode utilise `complete_prompt()` pour appeler directement le service d'IA.

In [4]:
# Cell 4: Plugin EmailGenerator

from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function

class EmailGeneratorPlugin:
    def __init__(self, state: EmailState, kernel: Kernel):
        self.state = state
        self.kernel = kernel
        # Ne pas créer la fonction sémantique dans le constructeur

    @kernel_function(
        name="generate_draft",
        description="Génère le brouillon de l'email en fonction des informations fournies"
    )
    async def generate_draft(self) -> str:
        # Création du prompt pour la génération d'email
        prompt = f"""
        Génère un email de type {self.state.type} à {self.state.recipient_name} ({self.state.recipient_role}) sur le sujet de {self.state.subject}.
        Les points clés à aborder sont : {', '.join(self.state.key_points)}.
        Le ton de l'email doit être {self.state.tone}.
        """
        if self.state.persona:
            prompt += f" Utilise le style d'écriture du persona {self.state.persona}."
        prompt += """
        L'email doit être bien structuré, clair et concis.
        """

        # Appel direct au service d'IA
        completion_service = self.kernel.get_service("default")
        result = await completion_service.complete_prompt(prompt)

        self.state.draft = str(result)
        return self.state.draft

## 5. Creation des Kernels et des Agents

Cette cellule configure les deux agents :
- **InputCollector** : Kernel avec le plugin InputCollector, instructions pour collecter les informations systematiquement
- **EmailGenerator** : Kernel avec le plugin EmailGenerator, instructions pour generer et proposer des modifications

Chaque agent possede son propre Kernel mais partage le meme `EmailState`.

In [5]:
# Cell 5: Création des Kernels et des Agents

from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent

# Création de l'état partagé
shared_state = EmailState()

# Kernel 1 (Input Collector)
input_kernel = sk.Kernel()
add_openai_service(input_kernel)
input_collector_plugin = InputCollectorPlugin(shared_state)
input_kernel.add_plugin(input_collector_plugin, "input_plugin")

input_agent = ChatCompletionAgent(
    kernel=input_kernel,
    name="InputCollector",
    instructions="""Vous êtes un assistant qui collecte des informations pour générer un email.

    IMPORTANT: Vous devez interagir directement avec l'utilisateur humain en posant des questions claires, une à la fois.
    Pour poser une question à l'utilisateur, utilisez TOUJOURS la fonction ask_human.

    Collectez systématiquement les informations suivantes dans cet ordre:
    1. Le type d'email (professionnel, amical, etc.)
    2. Le nom du destinataire
    3. Le rôle du destinataire (si applicable)
    4. Le sujet de l'email
    5. Au moins un point clé à aborder (demandez s'il y en a d'autres)
    6. Le ton souhaité (formel, informel, etc.)

    Après chaque réponse de l'utilisateur, utilisez la fonction appropriée pour enregistrer l'information.
    Utilisez get_state_summary régulièrement pour vérifier l'état actuel.
    Utilisez check_completeness pour vérifier si toutes les informations nécessaires sont disponibles.

    Une fois toutes les informations collectées, informez l'utilisateur que vous avez tout ce qu'il faut
    et que vous allez passer à la génération de l'email."""
)

# Kernel 2 (Email Generator)
email_kernel = sk.Kernel()
add_openai_service(email_kernel)
email_generator_plugin = EmailGeneratorPlugin(shared_state, email_kernel)
email_kernel.add_plugin(email_generator_plugin, "email_plugin")

email_agent = ChatCompletionAgent(
    kernel=email_kernel,
    name="EmailGenerator",
    instructions="""Vous êtes un assistant qui génère des emails personnalisés.

    Lorsque toutes les informations nécessaires sont collectées, utilisez la fonction generate_draft
    pour créer un brouillon d'email adapté.

    Une fois l'email généré, présentez-le à l'utilisateur et demandez-lui s'il souhaite y apporter des modifications.
    Si l'utilisateur demande des modifications, aidez-le à ajuster le contenu de l'email.
    Si l'utilisateur est satisfait, remerciez-le et concluez la conversation.

    En cas d'erreur ou d'informations manquantes, expliquez clairement ce qui manque et demandez à l'InputCollector
    de compléter les informations."""
)

## 6. Orchestration et Groupe de discussion

Cette section configure l'orchestration :
- **EmailCompletionStrategy** : Termine la conversation quand `conversation_complete` est True
- **SequentialSelectionStrategy** : Les agents parlent dans l'ordre, en commencant par InputCollector
- **get_user_input()** : Fonction asynchrone pour capturer les reponses utilisateur via widgets

Le `AgentGroupChat` orchestre les deux agents avec ces strategies.

In [ ]:
# Cell 6: Orchestration et Workflow

from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.strategies import SequentialSelectionStrategy
from semantic_kernel.agents.strategies import TerminationStrategy
from pydantic import BaseModel
from IPython.display import display, clear_output
import ipywidgets as widgets
import asyncio

# Stratégie de terminaison basée sur l'état de la conversation
class EmailCompletionStrategy(TerminationStrategy, BaseModel):
    state: EmailState  # Utilisation de l'annotation de type Pydantic

    async def should_terminate(self, agent, history, cancellation_token=None) -> bool:
        return self.state.conversation_complete

# Création d'une stratégie de sélection qui commence avec l'InputCollector
selection_strategy = SequentialSelectionStrategy(initial_agent=input_agent)

# Création du groupe de discussion avec la stratégie de sélection
group_chat = AgentGroupChat(
    agents=[input_agent, email_agent],
    selection_strategy=selection_strategy,
    termination_strategy=EmailCompletionStrategy(state=shared_state)
)

# Fonction asynchrone pour obtenir l'entrée utilisateur via ipywidgets (compatible Jupyter local)
async def get_user_input():
    """
    Obtient une entrée utilisateur de manière asynchrone en utilisant ipywidgets.
    Compatible avec Jupyter Notebook/Lab local (pas de dépendance Google Colab).
    """
    response_received = asyncio.Event()
    response_value = [None]

    # Création des widgets
    input_widget = widgets.Text(
        placeholder='Entrez votre réponse...',
        description='Réponse:',
        layout=widgets.Layout(width='400px')
    )
    submit_button = widgets.Button(
        description='Envoyer',
        button_style='primary'
    )
    output_widget = widgets.Output()

    # Callback pour le bouton
    def on_submit(b):
        with output_widget:
            clear_output()
            if input_widget.value.strip():
                response_value[0] = input_widget.value.strip()
                print(f"Reponse enregistree: {response_value[0]}")
                response_received.set()
            else:
                print("Veuillez entrer une reponse.")

    # Callback pour la touche Entrée
    def on_enter(change):
        if change['type'] == 'change' and change['name'] == 'value':
            pass  # On utilise on_submit pour le traitement

    submit_button.on_click(on_submit)

    # Permettre aussi la soumission avec Entrée (via un bouton caché qui se déclenche)
    def handle_submit(sender):
        on_submit(None)

    input_widget.on_submit(handle_submit)

    # Affichage des widgets
    display(widgets.HBox([input_widget, submit_button]))
    display(output_widget)

    # Attendre la réponse avec timeout
    try:
        await asyncio.wait_for(response_received.wait(), timeout=600)  # 10 minutes timeout
        return response_value[0]
    except asyncio.TimeoutError:
        print("Aucune reponse recue dans le delai imparti (10 minutes).")
        return "Timeout"

## 7. Fonction principale du workflow

La fonction `email_workflow()` orchestre tout le processus :
1. Initialise la conversation avec un message utilisateur
2. Itere sur les messages des agents
3. Detecte quand une question est posee et attend la reponse utilisateur
4. Affiche le brouillon genere et demande confirmation
5. Permet des modifications iteratives jusqu'a satisfaction

Le workflow gere les erreurs et affiche l'email final.

In [11]:
# Cellule 7: Fonction principale du workflow

async def email_workflow():
    try:
        # Initialisation de la conversation
        group_chat.history.add_user_message("Je voudrais créer un email personnalisé.")

        # Exécution de la conversation
        async for message in group_chat.invoke():
            # Afficher le message de l'agent
            print(f"[{message.name}] {message.content}")

            # Vérifier si un input humain est demandé
            if shared_state.human_input_requested:
                # Afficher la question
                print(f"\nQuestion: {shared_state.current_question}")

                # Obtenir la réponse de l'utilisateur
                user_response = await get_user_input()
                print(f"Réponse utilisateur reçue: {user_response}")

                # Ajouter la réponse à l'historique de la conversation
                group_chat.history.add_user_message(user_response)

                # Réinitialiser le flag
                shared_state.human_input_requested = False
                shared_state.current_question = ""

            # Si le brouillon est généré, l'afficher
            if shared_state.draft and not shared_state.conversation_complete:
                print("\n--- Brouillon de l'email : ---")
                print(shared_state.draft)
                print("\n--- Fin du brouillon ---")

                # Demander à l'utilisateur s'il est satisfait
                print("\nÊtes-vous satisfait de ce brouillon? (oui/non)")
                satisfied = await get_user_input()

                if satisfied.lower() in ["oui", "yes", "o", "y"]:
                    group_chat.history.add_user_message("Oui, je suis satisfait de ce brouillon.")
                    # Marquer la conversation comme terminée
                    shared_state.conversation_complete = True
                else:
                    # Demander des modifications
                    print("\nQuelles modifications souhaitez-vous apporter?")
                    modifications = await get_user_input()
                    group_chat.history.add_user_message(f"Je voudrais apporter les modifications suivantes: {modifications}")

        # Afficher le message final si la conversation est terminée
        if shared_state.conversation_complete:
            print("\n--- Email final : ---")
            print(shared_state.draft)
            print("\n--- Fin de l'email ---")
            print("\nMerci d'avoir utilisé notre service de création d'email!")

    except Exception as e:
        print(f"Erreur: {e}")
        import traceback
        traceback.print_exc()

## 8. Execution du workflow

Executez la cellule ci-dessous pour demarrer le processus interactif de creation d'email.

**Fonctionnement** :
1. L'agent InputCollector pose des questions une par une
2. Vous repondez via les widgets interactifs
3. Une fois toutes les informations collectees, l'agent EmailGenerator cree un brouillon
4. Vous pouvez demander des modifications ou valider l'email final

**Note** : Ce notebook necessite une interaction utilisateur via ipywidgets. Assurez-vous d'executer le notebook localement dans Jupyter Notebook ou JupyterLab.

In [ ]:
# Cellule 8: Exécution du workflow
# Exécutez cette cellule pour démarrer le processus de création d'email

# Exécution du workflow
await email_workflow()

## Conclusion

Ce notebook a illustre un workflow multi-agents complet pour la creation d'emails personnalises.

### Concepts techniques demontres

1. **Plugins personnalises** : `InputCollectorPlugin` et `EmailGeneratorPlugin` exposent des fonctions appelables par les agents
2. **Etat partage** : La classe `EmailState` (Pydantic) permet aux deux agents de collaborer sur les memes donnees
3. **Strategie de selection** : `SequentialSelectionStrategy` orchestre l'ordre de prise de parole
4. **Strategie de terminaison** : `EmailCompletionStrategy` detecte quand le workflow est termine
5. **Integration UI** : ipywidgets pour l'interaction utilisateur asynchrone

### Applications pratiques

Ce pattern peut etre adapte pour :
- **Assistants de redaction** : Documents, rapports, presentations
- **Chatbots structurants** : Collecte d'informations methodique pour formulaires complexes
- **Systemes de recommendation** : Collecte de preferences puis generation de suggestions
- **Workflows multi-etapes** : Tout processus necessitant validation iterative

### Points d'amelioration possibles

- Ajouter une validation des entrees (regex pour emails, longueur des textes)
- Implementer un systeme de templates d'emails pre-definis
- Sauvegarder l'historique des emails generes
- Ajouter un agent de review pour verifier la grammaire et le style
- Integrer avec une API d'envoi d'emails (SendGrid, Mailgun)